# Analysis Script

This script is used to regenerate all graphs reported in the paper "Predictable Accelerator Design with Time-Sensitive Affine types". The script uses data committed in the repository to generate the results. It **does not** regenerate the data.

There are four graphs in the paper.

1. Figure 4: Sensitivity analysis of unrolling and partitioning.
2. Figure 7: Exhaustive design space exploration for gemm-blocked.
3. Figure 8: Qualitative study of MachSuite benchmarks.
4. Figure 9: Resource Utilization for gemm-ncubed in Spatial.

The script will regenerate all the graphs. We welcome reviewers to perform further analysis/explore the data collected in the paper.

-----------

### Exhaustive Design Space Exploration (fig. 8)

This experiment collected data for 32,000 configurations of a blocked matrix-matrix multiply kernel. We partition
the data into three sets:

1. Set of Pareto-optimal points.
2. Set of points accepted by Dahlia.
3. Set of points that are Pareto optimal and accepted by Dahlia.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib
from benchmarking.plotting import plots
%matplotlib inline

# Location for all figures
FIG_DIR = "all-figures/"
# DPI setting
DPI = 300

In [ ]:
# Collected data.
exhaustive_dse_points = pd.read_csv('exhaustive-dse/data/summary.csv')

# Names of configurations accepted by Dahlia.
exhaustive_dahlia_accepted = pd.read_csv('exhaustive-dse/data/dahlia-points.csv')

# Remove prefix stem from benchmark names
exhaustive_dse_points.bench = \
    exhaustive_dse_points.apply(lambda row: row.bench.replace('gemm-dse:gemm-', ''), axis=1)
exhaustive_dahlia_accepted = \
    exhaustive_dahlia_accepted.apply(lambda row: row.bench.replace('dahlia-gemm-', ''), axis=1)

# Coerce columns into numeric types
for key in exhaustive_dse_points.columns:
    if key != 'bench':
        exhaustive_dse_points[key] = pd.to_numeric(exhaustive_dse_points[key], errors='coerce')

# Remove invalid rows.
exhaustive_dse_points = exhaustive_dse_points[exhaustive_dse_points.notnull().all(axis=1)].reset_index()

# Sanity check: latency estimates are tight.
for idx in range(len(exhaustive_dse_points)):
    assert exhaustive_dse_points.est_min_lat[idx] == exhaustive_dse_points.est_max_lat[idx], idx

In [ ]:
exhaustive_dahlia_points = \
  exhaustive_dse_points[exhaustive_dse_points["bench"].isin(exhaustive_dahlia_accepted.to_numpy())]

In [ ]:
# Resources to calculate pareto front for.
resources = [ 'est_' + key for key in [ 'ff', 'dsp', 'bram', 'lut' ]]

In [ ]:
# Calculate pareto points
# From: https://stackoverflow.com/questions/32791911/fast-calculation-of-pareto-front-in-python

def find_pareto(costs):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :return: A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    """
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    for i, c in enumerate(costs):
        (before, after) = costs[:i], costs[i+1:]
        # All points such that there is no point which has at least one objective minimized better and other objectives at least equal.
        is_efficient[i] = np.all(np.logical_not(np.logical_and(np.all(before <= c, axis=1), np.any(before < c, axis=1)))) and \
                          np.all(np.logical_not(np.logical_and(np.all(after <= c, axis=1), np.any(after < c, axis=1))))
    return is_efficient

def get_pareto():
    opts = find_pareto(exhaustive_dse_points[['est_compute_lat'] + resources].to_numpy())
    opt_idxs = np.where(opts == True)
    return exhaustive_dse_points.iloc[opt_idxs]


exhaustive_pareto_points = get_pareto()

# Number of true pareto trade off points
print("[Exhaustive DSE] Number of Pareto points: ", len(exhaustive_pareto_points[['est_compute_lat'] + resources].drop_duplicates()))

In [ ]:
# Dahlia points not in pareto frontier
def get_dahlia_pareto():    
    non_pareto_dahlia = set(exhaustive_dahlia_points.bench).difference(set(exhaustive_pareto_points.bench))
    pareto_dahlia = set(exhaustive_dahlia_points.bench).intersection(set(exhaustive_pareto_points.bench))
    return (
        exhaustive_dse_points[exhaustive_dse_points.bench.isin(non_pareto_dahlia)],
        exhaustive_dse_points[exhaustive_dse_points.bench.isin(pareto_dahlia)]
    )

exhaustive_non_pareto_dahlia_points, exhaustive_pareto_dahlia_points = get_dahlia_pareto()

print("[Exhaustive DSE] Dahlia non-pareto configurations: ", 
      len(exhaustive_non_pareto_dahlia_points[['est_compute_lat'] + resources].drop_duplicates()))
print("[Exhaustive DSE] Dahlia pareto configurations: ", 
      len(exhaustive_pareto_dahlia_points[['est_compute_lat'] + resources].drop_duplicates()))

In [ ]:
def plot_exhaustive():
    sns.set()

    pal = sns.color_palette('colorblind', 8)

    # Settings
    groups = [
        {'label': 'All', 'data': exhaustive_dse_points, 'color': pal[-1], 'marker': 'o', 'alpha': 0.05},
        {'label': 'Pareto', 'data': exhaustive_pareto_points, 'color': pal[1], 'marker': 's', 'alpha': 0.5},
        {'label': 'Dahlia', 'data': exhaustive_dahlia_points, 'color': pal[2], 'marker': 'X', 'alpha': 0.2},
        {'label': 'Non pareto Dahlia', 'data': exhaustive_non_pareto_dahlia_points, 'color': pal[2], 'marker': 'X', 'alpha': 0.2},
        {'label': 'Pareto points accepted by Dahlia', 'data': exhaustive_pareto_dahlia_points, 'color': pal[2], 'marker': 'X', 'alpha': 0.2}
    ]

    to_plot = [
        {'prefix': 'only-pareto', 'groups': [0, 1]},
        {'prefix': 'dahlia-pareto', 'groups': [0, 1, 4]},
        {'prefix': 'dahlia-non-pareto', 'groups': [0,3]}
    ]

    for plot_info in to_plot:
        fig = plt.figure()
        ax = fig.gca()
        for g in plot_info['groups']:
            group = groups[g]
            sns.scatterplot(x='est_compute_lat', 
                            y='est_lut', data=group['data'], 
                            color=group['color'], 
                            marker=group['marker'],
                            label=group['label'],
                            rasterized=True)

        ax.get_xaxis().set_major_formatter(
            matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x / 10 ** 6), ',')))
        ax.get_yaxis().set_major_formatter(
            matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
        ax.set_xlabel('Estimated latency (millions of cycles)')
        ax.set_ylabel('Estimate LUTs')
        fig.tight_layout()
        name = FIG_DIR + "gemm-blocked-dse-{}.pdf".format(plot_info['prefix'])
        print("[Exhaustive DSE] Generating " + name)
        fig.savefig(name, dpi=DPI)
        
plot_exhaustive()

----

### Sensitivity Analysis (Fig 4)

Figure 4 in Section 2 analyzes the effects of varying unrolling and partitioning independently of each other. There are three figures:

1. Varying unrolling with no partitioning.
2. Varying unrolling with constant partitioning.
3. Varying unrolling and paritioning in lockstep.

In [ ]:
# Data locations

sensitivity_no_part = \
  pd.read_csv('./sensitivity-analysis/no-partition-unroll/summary.csv').sort_values(by='unroll').reset_index()
sensitivity_const_part = \
  pd.read_csv('./sensitivity-analysis/const-partition-unroll/summary.csv').sort_values(by='unroll').reset_index()
sensitivity_lockstep = \
  pd.read_csv('./sensitivity-analysis/lockstep-partition-and-unroll/summary.csv').sort_values(by='partition').reset_index()

In [ ]:
def make_sensitivity_plots():
    # No partition plot.
    print('[Sensitivity Analysis] Generating no partition graphs.')
    plots.make_absolute_plots(
        sensitivity_no_part,
        x_key = 'unroll',
        y_keys = ['runtime_avg', 'lut_used'],
        group_by = lambda x: 0,
        x_label = 'Unrolling factor (no partitioning)',
        fig_prefix = 'no-partition-unrolling',
        dpi = DPI,
        fig_dir = FIG_DIR,
        )
    
    # Constant partitioning
    print('[Sensitivity Analysis] Generating constant partition graphs.')
    plots.make_sec2_plot(
        df=sensitivity_const_part,
        x_key='unroll',
        x_label = 'Unrolling factor (partitioning = 8)',
        factor = 8,
        fig_prefix = 'const-partition-unroll',
        legend = 'lut_used',
        dpi = DPI,
        fig_dir = FIG_DIR,
    )
    
    # Lockstep partitioning and unrolling
    print('[Sensitivity Analysis] Generating lockstep partition graphs.')
    plots.make_sec2_plot(
        df=sensitivity_lockstep,
        x_key = 'partition',
        x_label = 'Partitioning and Unrolling factor',
        fig_prefix = 'const-len-partition-and-unroll',
        dpi=DPI,
        fig_dir = FIG_DIR)


make_sensitivity_plots()

-------

### Qualitative Evaluation (fig 8)

The qualitative evaluation consists of parameter sweeps over three MachSuite benchmarks:

1. stencil-2d
2. md-knn
3. md-grid

For each benchmark, we generated several configurations and ran estimation for the benchmarks accepted by Dahlia. We **did not** run estimation for all possible configurations--just the ones accepted by Dahlia.

In [ ]:
SCALE_X = {"scale_fun": lambda x: float(x / 100), "scale_label": "Hundreds of cycles"}

RESOURCES = ['bram_used', 'dsp48_used', 'ff_used', 'lut_used', 'avg_latency']

In [ ]:
# Benchmark files
qual_benchs = ['md-knn-summary', 'md-grid-summary', 'stencil2d-inner-summary']

# Dataframes with all the data.
qual_dfs = { bench_name : pd.read_csv(f"qualitative-study/data/{bench_name}.csv") for bench_name in qual_benchs }

# Calculate average latencies for each benchmark.
for key in qual_benchs:
    qual_dfs[key]['avg_latency'] = ((qual_dfs[key]['min_latency'] + qual_dfs[key]['max_latency']) /  2)

In [ ]:
def plot_stencil2d():
    df = qual_dfs['stencil2d-inner-summary']
    res = 'ur_loop2'

    opts = find_pareto(df[RESOURCES].to_numpy())
    pareto = df.iloc[opts]
    
    print(f'[Qualitative Study] Accepted Pareto points for stencil2d: {len(pareto)}/{len(df)}')

    def group(idx):
        return df.iloc[idx][res]

    plots.make_qual_plot(
        pareto,
        x_data = {'key': 'avg_latency', 'label': 'Average Latency'},
        y_data = {'key': 'lut_used', 'label': 'LUTs Used'},
        group_by = group,
        legend="Inner Unroll",
        fig_prefix = "stencil-inner-2d-inner-unroll",
        scale_x = SCALE_X,
        fig_dir = FIG_DIR,
        dpi=DPI,
    )
    
plot_stencil2d()

## Removing outliers from md_knn

In the graph above, the points on the extreme right trade-off a lot of latency for reducing the LUT count by a bit. In the paper, we decided to not show these points **but we explicitly call this out in the prose of the paper**.

In [ ]:
def plot_md_knn():
    df = qual_dfs['md-knn-summary']
    df = df[df.avg_latency < 100000].reset_index()
    res = 'ur_loop1'

    opts = find_pareto(df[RESOURCES].to_numpy())
    pareto = df.iloc[opts]
    
    print(f'[Qualitative Study] Accepted Pareto points for md-knn: {len(pareto)}/{len(df)}')
    
    def group(idx):
        return df.iloc[idx][res]
            
    plots.make_qual_plot(
        pareto,
        x_data = {'key': 'avg_latency', 'label': 'Average Latency'},
        y_data = {'key': 'lut_used', 'label': 'LUTs Used'},
        group_by = group,
        legend='Outer Unroll',
        fig_prefix = "md-knn-outer-unroll",
        scale_x = SCALE_X,
        fig_dir = FIG_DIR,
        dpi=DPI,
    )

plot_md_knn()

In [ ]:
def plot_md_grid():
    df = qual_dfs['md-grid-summary']
    res = 'B0Y_UR'

    opts = find_pareto(df[RESOURCES].to_numpy())
    pareto = df.iloc[opts]
    
    print(f'[Qualitative Study] Accepted Pareto points for md-grid: {len(pareto)}/{len(df)}')

    def group(idx):
        return df.iloc[idx][res]

    plots.make_qual_plot(
        pareto,
        x_data = {'key': 'avg_latency', 'label': 'Average Latency'},
        y_data = {'key': 'lut_used', 'label': 'LUTs Used'},
        group_by = group,
        legend="Middle Unroll",
        fig_prefix = "md-grid-middle-unroll",
        scale_x = SCALE_X,
        fig_dir = FIG_DIR,
        dpi=DPI,
    )
    
plot_md_grid()

----------

### Spatial Comparison (fig. 9)

Spatial is a high level programming language for designing hardware. In our paper, we sweep over parameters for a general matrix multiply design in Spatial and demonstrate that Spatial automatic parameter inference for banking and unrolling can also generate unpredictable designs.

In [ ]:

def plot_spatial_comp():
    pal = sns.color_palette("colorblind",6)

    PERF_RESOURCES = ['dsp_used', 'bram_tile_used','lut_used']

    df = pd.read_csv('spatial-sweep/data/resource_summary.csv')

    # Normalize resource usage against the no unrolling case.
    for res in PERF_RESOURCES:
        baseline = df[res][0]
        df[res + "_norm"] = df[res] / baseline
            
    fig = plt.figure()
    ax = fig.gca()
    loop_k = np.arange(16)+1
    maker = ['X','D','o']
    for i, res in enumerate(PERF_RESOURCES):
        y = df[res + '_norm']
        sns.lineplot(loop_k, y, linewidth=3, marker=maker[i], ms=10)
       

    plt.xticks(fontsize = 14)
    plt.yticks(fontsize = 14)
    ax.set_ylabel('Normalized Resource Usages',fontsize= 16)
    ax.set_xlabel('Unrolling Factor',fontsize = 16)
    
    PERF_RESOURCES_NAME = ['DSP used', 'BRAM used', 'LUT used']
    ax.legend(PERF_RESOURCES_NAME, prop={'size': 14})
    fig.tight_layout()
    plt.savefig(FIG_DIR + 'paper-normalized.pdf', bbox_inches='tight', dpi=DPI)


plot_spatial_comp()

----

### Vega Graphs

In [ ]:
import altair as alt
from altair import datum
from altair_saver import save

pareto = exhaustive_pareto_points.bench
dahlia = exhaustive_dahlia_points.bench
dahlia_pareto = exhaustive_pareto_points[exhaustive_pareto_points.bench.isin(dahlia)].bench

pareto_label = 'Pareto'
dahlia_label = 'Dahlia'
dp_label = 'Dahlia & Pareto'

df = exhaustive_dse_points.copy()
df['kind'] = 'All'
df.loc[df.bench.isin(pareto), 'kind'] = pareto_label
df.loc[df.bench.isin(dahlia), 'kind'] = dahlia_label
df.loc[df.bench.isin(dahlia_pareto), 'kind'] = dp_label
# Counts for all kinds\
df = df[['est_compute_lat', 'kind'] + resources].drop_duplicates()
print(alt.__version__)

In [ ]:
alt.data_transformers.enable('data_server')

chart = alt.Chart(df).mark_point(size=40, filled=True).encode(
    x=alt.X(
        axis=alt.Axis(format='~s', title='Latency (cycles)', tickCount=8),
        field='est_compute_lat', 
        type='quantitative',
    ),
    y=alt.Y(
        axis=alt.Axis(format='~s', title=None, tickCount=5),
        field='est_lut',
        type='quantitative',
    ),
    color=alt.Color(
        'kind',
        # legend=None,
        scale=alt.Scale(
            domain=['All', dp_label, dahlia_label, pareto_label],
            range=['#c5c5c5', '#1b9e77', '#d95f02', '#7570b3'])
    ),
    opacity=alt.Opacity(
        'kind',
        # legend=None,
        scale=alt.Scale(
            domain=['All', dp_label, dahlia_label, pareto_label],
            range=[0.3, 1, 1, 1])
    ),
    shape=alt.Opacity(
        'kind',
        # legend=None,
        scale=alt.Scale(
            domain=['All', dp_label, dahlia_label, pareto_label],
            range=['circle', 'diamond', 'circle', 'circle'])
    ),
).properties(
    width=450,
)


def set_style(chart, suppress_y=False):
    return chart.configure_view(
        strokeWidth=0
    ).configure_legend(
        titleFontSize=15,
        labelFontSize=14,
        title=None,
        orient='top-right',
        # disable=True,
    ).configure_axis(
        grid=False,
        labelFontSize=15,
        titleFontSize=15,
    )

In [ ]:
base = chart.mark_circle(size=15).encode(
    y=alt.Y(
        axis=alt.Axis(format='~s', title='LUTs used', tickCount=5),
        field='est_lut',
        type='quantitative',
    ),
    color=alt.Color(
        'kind',
        scale=alt.Scale(
            domain=['All', dp_label, dahlia_label, pareto_label],
            range=['#c5c5c5', '#1b9e77', '#d95f02', '#7570b3'])
    ),
)

fig1 = base.transform_filter((datum.kind == 'All')) + \
    base.mark_circle(size=20).transform_filter((datum.kind == pareto_label))

res = set_style(fig1)
res.save('dse-pareto.svg')
res

In [ ]:
fig2 = chart.mark_circle(size=15).transform_filter((datum.kind == 'All')) + \
    chart.mark_circle(size=20).transform_filter((datum.kind == dahlia_label))


res = set_style(fig2)
res.save('dse-dahlia.svg')
res

In [ ]:
small = chart.transform_filter((datum.est_compute_lat < 1600000) & (datum.est_lut < 40000)).encode(
    x=alt.X(
        axis=alt.Axis(format='~s', title='Latency (cycles)'),
        field='est_compute_lat', 
        type='quantitative',
    ),
)

fig3 = small.transform_filter((datum.kind == 'All')) + \
    small.transform_filter((datum.kind != "All")) + \
    small.mark_circle(size=60).transform_filter((datum.kind == dp_label))

res = set_style(fig3)
res.save('dse-zoom.svg')
res

In [ ]:
qual_dfs.keys()

In [ ]:
stencil_2d = qual_dfs['stencil2d-inner-summary']
s2d = stencil_2d.iloc[find_pareto(stencil_2d[RESOURCES].to_numpy())]
s2d_chart = alt.Chart(s2d).mark_point(size=200, filled=True).encode(
    x=alt.X(
        axis=alt.Axis(format='~s', title='Latency (cycles)', tickCount=8),
        field='avg_latency', 
        type='quantitative',
        scale=alt.Scale(domain=(50000,300000))
    ),
    y=alt.Y(
        axis=alt.Axis(format='~s', title = "LUTs used", tickCount=5),
        field='lut_used',
        type='quantitative',
        scale=alt.Scale(domain=(2000,6000))
    ),
    color=alt.Color(
        'ur_loop2',
        legend=alt.Legend(title="Inner unroll", orient='top-right'),
        scale=alt.Scale(
            domain=[1, 3],
            range=['#1b9e77', '#d95f02']),
        type='ordinal'
    ),
).properties(
    width=400,
)

res = set_style(s2d_chart)
res.save('stencil2d-inner.svg')
res

In [ ]:
md_knn = qual_dfs['md-knn-summary']
md_knn_p = md_knn.iloc[find_pareto(md_knn[RESOURCES].to_numpy())]
md_knn_chart = alt.Chart(md_knn_p).mark_point(size=200, filled=True).encode(
    x=alt.X(
        axis=alt.Axis(format='~s', title='Latency (cycles)', tickCount=3),
        field='avg_latency', 
        type='quantitative',
        scale=alt.Scale(domain=(16000,18200))
    ),
    y=alt.Y(
        axis=alt.Axis(format='~s', title = None, tickCount=5),
        field='lut_used',
        type='quantitative',
    ),
    color=alt.Color(
        'ur_loop1',
        legend=None,
        scale=alt.Scale(scheme = 'dark2'),
        type='ordinal'
    ),
).properties(
    width=220,
)

alt.Resolve(scale=alt.LegendResolveMap(color=alt.ResolveMode('independent')))

md_knn_both = (md_knn_chart.transform_filter(datum.avg_latency < 100000) | \
md_knn_chart.transform_filter(datum.avg_latency > 100000).encode(
    x=alt.X(
        axis=alt.Axis(format='~s', title='Latency (cycles)', tickCount=3),
        field='avg_latency', 
        type='quantitative',
        scale=alt.Scale(domain=(128550,128800))
    ),
    y=alt.Y(
        axis=alt.Axis(format='~s', title = None, tickCount=5),
        field='lut_used',
        type='quantitative',
        scale=alt.Scale(domain=(8000,12000))
    ),
    color=alt.Color(
        'ur_loop1',
        legend=alt.Legend(title="Outer unroll", orient='top-right'),
        scale=alt.Scale(scheme = 'dark2'),
        type='ordinal'
    ),

)).resolve_legend('independent')#.resolve_axis(x='shared')
res = set_style(md_knn_both)
res.save('md-knn.svg')
res

In [ ]:
md_grid = qual_dfs['md-grid-summary']
md_grid_p = md_grid.iloc[find_pareto(md_grid[RESOURCES].to_numpy())]
md_grid_chart = alt.Chart(md_grid_p).mark_point(size=200, filled=True).encode(
    x=alt.X(
        axis=alt.Axis(title='Latency (cycles)', tickCount=3),
        field='avg_latency', 
        type='quantitative',
        scale=alt.Scale(domain=(7940,8045))
    ),
    y=alt.Y(
        axis=alt.Axis(format="~s", title = None, tickCount=6),
        field='lut_used',
        type='quantitative',
        scale=alt.Scale(domain=(10000,50000))
    ),
    color=alt.Color(
        'B0Y_UR',
        legend=alt.Legend(title="Middle unroll", orient='top-right'),
        scale=alt.Scale(
            scheme = 'dark2',
        ),
        type='ordinal'
    ),
).properties(
    width=400,
)

res = set_style(md_grid_chart)
res.save('md-grid.svg')
res